In [11]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

from flashtext import KeywordProcessor
# from groupby_toolz import enrich_db, gcloud
# pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500

In [12]:
from groupby_dsa import dsa

In [13]:
# parameters
customer_id = '115'
formatted_attribute = 'Concern'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Timing Belt"""

regex_pattern = r"(?i)(max)(min)"
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%lovelyskin%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [14]:
metric = ' Inches'

range_string = """
 0 - 0.5000 Inches
 0 - 10 Inches
 0.5001 - 1.0000 Inches
 0 - 5 Inches
 1.0001 - 1.5000 Inches
 11 - 15 Inches
 11 - 20 Inches
 1.5001 - 2.0000 Inches
 16 - 20 Inches
 2.0001 - 2.5000 Inches
 21 - 25 Inches
 21 - 30 Inches
 2.5001 - 3.0000 Inches
 26 - 30 Inches
 3.0001 - 3.5000 Inches
 31 - 40 Inches
 3.5001 - 4.0000 Inches
 4.0001 - 4.5000 Inches
 41 - 50 Inches
 4.5001 - 5.0000 Inches
 6 - 10 Inches
 Greater Than 30 Inches
 Greater than 5.0000 Inches
 Greater than 50 Inches
 """

In [15]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [18]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df

bucket attributes external_id  \
0      Arbors, Bowers, & Pergolas     [Care]    69659382   
1      Arbors, Bowers, & Pergolas     [Care]    69659384   
2      Arbors, Bowers, & Pergolas     [Care]    69660258   
3      Arbors, Bowers, & Pergolas     [Care]    69660259   
4      Arbors, Bowers, & Pergolas     [Care]    69660260   
...                           ...        ...         ...   
72457             Wrapping Papers    [Width]    65030868   
72458             Wrapping Papers    [Width]    65030875   
72459             Wrapping Papers    [Width]    65030882   
72460             Wrapping Papers    [Width]    65051979   
72461             Wrapping Papers    [Width]    65051993   

                                                                                       name  \
0      New England Arbors® Concord 4-Foot 6-Inch x 8-Foot Composite Garden Trellis in Brown   
1              New England Arbors® Heartwood 5-Foot 6-Inch x 6-Foot 8-Inch Trellis in Brown   
2          New England Arbors® Nantucket Legacy 5-Foot x 8-Foot 6-Inch Vinyl Arbor in White   
3                                                Fairfield 12'10 x 7'2 Vinyl Arbor in White   
4                                           Fairfield Grande 9'3 x 7'2 Vinyl Arbor in White   
...                                                                                     ...   
72457                                              25-Pack Winter Wonderland Gift Wrap Case   
72458                                                  25-Pack Winter Whimsy Gift Wrap Case   
72459                                               25-Pack Tropical Holiday Gift Wrap Case   
72460                                                           25-Pack Glam Gift Wrap Case   
72461                                                  25-Pack Feliz Navidad Gift Wrap Case   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  description  
0                                                                                                                                                                                                                                                                                               The New England Arbors Hamilton Trellis is expertly crafted to keep your plants in line. Designed with distinct details, the backyard staple features a curved semi-circle top with creative brown finish for a unique outdoor accent. Wonderfully designed to act as a dynamic accent wall or privacy screen Semi-circle top and lattice design ideal for use in your garden Simple assembly required Tools required for permanent in-ground installation Soft brown finish Crafted with premium weather-resistant composite for outdoor use Wipe clean Measures 1.5\" L x 39\" W x 77.25\" H Weighs 36 lb. 10-year limited manufacturer's warranty Imported  
1      Keep your backyard booming with the New England Arbors Heartwood Trellis. Designed with natural inspired cedar, the privacy screen features a large grid to support your blooming plants and allow them to reach newer heights. The Heartwood Cedar Trellis is a subtle and modern re-imagining of traditional

In [19]:
df['attributes'].explode().value_counts()

Occasion                      31522
Personalization Type          31475
Material                       1041
Color                          1006
Origin                          848
Features                        714
Care                            581
Width                           577
Length                          571
Height                          474
Weight                          342
Indoor/Outdoor                  264
Package Quantity                218
Diameter                        171
Pattern                         136
Battery Type                    135
Weave                           122
Cord Length                      86
Sport                            86
Fabric Weight Type               80
Power Type                       74
Weight Capacity                  70
Concern                          69
Count                            69
Backing Material                 67
Size Length Rugs/Runners         67
Size Diameter Rugs/Runners       67
Size Width Rugs/Runners     

In [ ]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

In [ ]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [39]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [40]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [41]:
x

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, values, matches]
Index: []

In [ ]:
x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [ ]:
df[df['external_id'].astype(str)=='00739441']

In [ ]:
# x['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [ ]:
split_col(x)

In [ ]:
len(x['matches'].explode().value_counts())

In [ ]:
# error

In [ ]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

In [ ]:
x['matches']=x['matches'].astype(float)

In [ ]:
x

# matches

In [ ]:
matches=x
matches

In [ ]:
matches['matches']=matches['matches'].astype(float)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [ ]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

In [ ]:
matches['Q:outside_diameter'].explode().value_counts()

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [20]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [21]:
looks_good('Motion Industries', attribute, df, wipe)